In [1]:
import h5py
import pylab as pl
import numpy as np
import scipy.stats as sp
import matplotlib as mpl
import networkx

In [2]:
def split_unique_id(unique_id):
    """Splits the ids assign to the subhalos by the merger tree code by snap number and subfind number """
    subfind_number = int(unique_id % 1e6)
    snap_number = int((unique_id - subfind_number) / 1e6)
    
    return snap_number, subfind_number

def get_main_branch_unique_ids(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    i = 0
    while True:
        succesors = list(subtree.successors(node))
        if len(succesors) == 0:
            break
        node = succesors[0] # select only the first succesor (main branch)
        mpb.append(node)
        
    return mpb

In [3]:
def getNeaSnap(z, Zs, snapN):
    """Return the snapshot number closest to the redshfit z, where Zs and snapN are a list of redshifts and snapshots"""
    ii = np.searchsorted(Zs, z)
    if z - Zs[ii-1] < Zs[ii]-z:
        return snapN[ii-1]
    else:
        return snapN[ii]
    
def getFofId(fofZ0, z, tree, Zs, snapN):
    """return the snapshot and the fofid of the main progenitor of the fofz0 subgroup at redhsift z"""
    uniqueID0 = int(128*1e6+fofZ0)
    stree = networkx.dfs_tree(tree, str(uniqueID0))
    mtree = get_main_branch_unique_ids(stree, str(uniqueID0))
    snapZ = getNeaSnap(z, Zs, snapN)
    numberFoF, nsnapL = [], [] 
    for m in mtree:
        tsnapN, tsfid = split_unique_id(int(m))
        numberFoF.append(tsfid)
        nsnapL.append(tsnapN)
    numberFoF, nsnapL = np.array(numberFoF), np.array(nsnapL)
    iid = np.where(nsnapL==snapZ)
    if iid[0].size > 0:
        iid = iid[0][0]
    else:
        raise Exception('The subgroup {} has no progenitors at redshift {}'.format(fofZ0, z))
    return snapZ, numberFoF[iid]

In [4]:
# and the redhsift/snapnumber lists
sim = h5py.File('/data/cielo/simulations/LG1/LG1.hdf5', 'r')

Zs, snapN = [], []

for i in range(128, 109, -1):
    sname = 'SnapNumber_{}/Header/Redshift'.format(i)
    zt = sim[sname][()]
    Zs.append(zt)
    snapN.append(i)

Zs, snapN = np.array(Zs), np.array(snapN)

In [6]:
# and now for something completely different
# lets find all the halos and a list of subhalos
# if any of the offsets of gas particles is negative, i remove the subhalo from the list

GrouList = sim['SnapNumber_127/Groups/GroupNumber'][()]
SubGroupList = sim['SnapNumber_127/SubGroups/GroupNumber'][()]

SubGroupPass, SubGroupPassCentral = [], []
subGroupCentral_wS = []
subGroupCentral_woS = []

for g in GrouList:
    ii = np.where(SubGroupList==g)[0]
    if ii.size>1: # at least a satellite
        HalfMass = sim['SnapNumber_127/SubGroups/SubGroupHalfMass'][ii]
        jj = np.argsort(HalfMass) # idk if halfmass
        ii2 = ii[jj[:-1]] # all the index but the more massive
        iMM = ii[-1] # the most massive one
        ii2 = np.sort(ii2) # the index must be sorted for reading from a hdf5, idk why
        #OffG = sim['SnapNumber_127/SubGroups/PartType0/Offsets'][ii2]
        #jj2 = np.where((OffG[:,0]>=0) & (OffG[:,1]>=0))[0]
        #if jj2.size>0:# at least one:
        #    ii3 = ii2[jj2]
        #    SubGroupPass.extend(list(ii3))
        #    SubGroupPassCentral.extend([iMM]*ii3.size)
        SubGroupPass.extend(list(ii2)) # this is the version without checking if has particles at z=0
        SubGroupPassCentral.extend([iMM]*ii2.size)
        
        OffMM = sim['SnapNumber_128/SubGroups/PartType0/Offsets'][iMM] # this must be one set of offsets
        if OffMM[0]>=0 and OffMM[1]>=0:
            subGroupCentral_wS.append(iMM)
    elif ii.size==1: # just a central galaxy:
        OffOG = sim['SnapNumber_128/SubGroups/PartType0/Offsets'][ii[0]]
        if OffOG[0]>=0 and OffOG[1]>=0:
            subGroupCentral_woS.append(ii[0])
        

SubGroupPass, SubGroupPassCentral = np.array(SubGroupPass), np.array(SubGroupPassCentral)
subGroupCentral_wS = np.array(subGroupCentral_wS)
subGroupCentral_woS = np.array(subGroupCentral_woS)

In [7]:
SubGroupPass.size, SubGroupPassCentral.size, subGroupCentral_wS.size, subGroupCentral_woS.size

(1012, 1012, 10, 454)

In [8]:
# ok,lets find out if any of those had gas particles, or at least a progenitors, at z=2
trees = networkx.read_multiline_adjlist('/data/cielo/simulations/LG1/LG1_merger_trees.txt')

In [12]:
SubGroupPass2, SubGroupAtZ2 = [], []
zT = 0.2

for ss in SubGroupPass:
    try:
        snap, fofid = getFofId(ss, zT, trees, Zs, snapN)
        OffG = sim['SnapNumber_{}/SubGroups/PartType0/Offsets'.format(snap)][fofid]
        if OffG[0]>=0 and OffG[1]>=0:
            SubGroupPass2.append(ss)
            SubGroupAtZ2.append(fofid)
    except:
        pass
    
SubGroupPass2, SubGroupAtZ2 = np.array(SubGroupPass2), np.array(SubGroupAtZ2)

In [13]:
SubGroupPass2.size

14

In [11]:
np.unique(SubGroupPassCentral).size

49

In [10]:
uu = np.unique(SubGroupPassCentral)
c =0
for u in uu:
    Off = sim['SnapNumber_128/SubGroups/PartType0/Offsets'][u]
    if Off[0]>=0 and Off[1]>=0:
        c +=1
c

10

In [8]:
#I'm gonna write this down, as hdf5, I guess.
with h5py.File('SatelliteList.h5', 'w') as f:
    f.create_dataset('SatelliteList', data=SubGroupPass)
    f.create_dataset('SatelliteCentral', data=SubGroupPassCentral)
    #f.create_dataset('CentralLists/withSatellites', data=subGroupCentral_wS)
    #f.create_dataset('CentralLists/withoutSatellites', data=subGroupCentral_woS)

In [10]:
# and now for something completely different
# lets find all the halos and a list of subhalos
# if any of the offsets of gas particles is negative, i remove the subhalo from the list

SubGroupPassV2, SubGroupPassCentralV2 = [], []

for g in GrouList:
    ii = np.where(SubGroupList==g)[0]
    if ii.size>1: # at least a satellite
        ii2 = ii[1:] # all the index but the first
        iMM = ii[0] # the most massive one
        ii2 = np.sort(ii2) # the index must be sorted for reading from a hdf5, idk why
        SubGroupPassV2.extend(list(ii2)) # this is the version without checking if has particles at z=0
        SubGroupPassCentralV2.extend([iMM]*ii2.size)
        
SubGroupPassV2, SubGroupPassCentralV2 = np.array(SubGroupPass), np.array(SubGroupPassCentral)

In [11]:
SubGroupPassV2.size, SubGroupPassCentralV2.size

(1012, 1012)

In [13]:
SubGroupPassV2-SubGroupPass

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
SubGroupPassCentralV2-SubGroupPassCentral

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
np.where((SubGroupPassV2==SubGroupPass)==False)

(array([], dtype=int64),)